# IRIS Classification

In [1]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import svm

/opt/conda/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
iris = datasets.load_iris()
digits = datasets.load_digits()
X = iris.data
y = iris.target

# IRIS First Model Creation

In [3]:
clf = RandomForestClassifier(random_state=0, n_jobs=-1)

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2, random_state=0)
clf = clf.fit(xtrain, ytrain)

In [5]:
y_pred=clf.predict(xtest)
print(xtest.shape)
print(ytest.shape)
print(xtrain.shape)
print(ytrain.shape)

(30, 4)
(30,)
(120, 4)
(120,)


In [6]:
prob=clf.predict_proba(xtest)
#print(prob)
prob1=prob[:,1]

# Iris Second Model Creation

In [7]:
clf_2 = RandomForestClassifier(random_state=123, n_jobs=10)
xtrain2, xtest2, ytrain2, ytest2 = train_test_split(X,y,test_size=0.25, random_state=123)
clf_2 = clf_2.fit(xtrain2, ytrain2)
y_pred2=clf_2.predict(xtest2)
print(xtest2.shape)
print(ytest2.shape)
print(xtrain2.shape)
print(ytrain2.shape)
prob_2=clf_2.predict_proba(xtest2)[:,1]

(38, 4)
(38,)
(112, 4)
(112,)


# score 1st Model

In [8]:
@scoring_func
def score_1(model, request):
    import numpy
    payload = request.json["payload"]
    data_list = payload
    data_array = numpy.asarray(data_list)
    prediction = model.predict(data_array)
    return prediction.tolist()

# score 2nd Model

In [9]:
@scoring_func
def score_2(model, request):
    import numpy
    payload = request.json["payload"]
    data_list = payload
    data_array = numpy.asarray(data_list)
    prediction = model.predict(data_array)
    return prediction.tolist()

In [10]:
import requests
req = requests.Request()
req.json = {"payload":[[5.8, 2.8, 5.1, 2.4],
       [6.0 , 2.2, 4.0 , 1.0 ]]}

In [11]:
xtest[:2]

array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ]])

In [12]:
we=score_1(clf,req)

In [13]:
import numpy as np
per = np.array(we)

In [14]:
per*0.5

array([1. , 0.5])

In [15]:
xtest2[0]

array([6.3, 2.5, 4.9, 1.5])

In [16]:
import requests
req2 = requests.Request()
req2.json = {"payload":[[6.3, 2.5, 4.9, 1.5]]}

In [17]:
score_2(clf_2,req2)

[1]

# Registering 1st model

In [18]:
register_model(clf,
                score_1,
                "sklearn_iris_v11_a",
                "sklearn_iris_v11_a",
                MLModelFlavours.sklearn,
                input_type="json",
                explain_ai=False,
                x_train=xtrain, 
                y_train=ytrain,
                y_true=ytest,
                y_pred=y_pred,
                model_type="classification",
                prob=prob1,
                feature_names=iris.feature_names,
                feature_ids=iris.feature_names,
                target_names=iris.target_names.tolist()
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# Registering 2nd model

In [19]:
register_model(clf_2,
                score_2,
                "sklearn_iris_v22_a",
                "sklearn_iris_v22_a",
                MLModelFlavours.sklearn,
                input_type="json",
                explain_ai=False,
                x_train=xtrain2, 
                y_train=ytrain2,
                y_true=ytest2,
                y_pred=y_pred2,
                model_type="classification",
                prob=prob_2,
                feature_names=iris.feature_names,
                feature_ids=iris.feature_names,
                target_names=iris.target_names.tolist()
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
